In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import sklearn
import sys
from scipy.stats import norm
import warnings
from scipy.stats.stats import pearsonr
from pandas import ExcelWriter
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog
warnings.filterwarnings("ignore")

In [2]:
#Version 3.1 : Remove Dummy Entries Updated - MVK
#Version 3.2 : Browse Button Added

In [3]:
def clean(data):

    #REMOVE DUMMY ENTRIES ***Version 3.1 Update***
    
    data = data[~data['SubCategory'].str.contains("DummyCategory")]

    #DUPLICATE USERS CHECK
    dc = data[['UserName','QuestionId']]
    dups = dc[dc.duplicated()]
    
    if(dups.empty == False):
        print('\n\n\033[1mDuplicate values in data! Check the following usernames:\033[0m')
        print(set(dups['UserName']))
        sys.exit()


    pc = data.pivot_table(index = "UserName", columns = "QuestionId", values = ["OptionNumber", "TimeTaken"])
    pc.columns.names = (None, None)

    #REMOVE USERS BASED ON TIME
    for user, time in pc['TimeTaken'].iterrows():
        count = 0
        #questions = time.count()
        for i in time:
            if i < 2: count += 1
        if count>20: #Condition Subject to Change
            pc.drop(user, inplace=True)

    #SAME ANSWER CHECK
    for user, option in pc['OptionNumber'].iterrows():
        count = [0,0,0,0,0]
        thresh = 0.8*option.count() #Threshold Subject to Change

        for i in option:
            if i==1:
                count[0] += 1
            if i==2:
                count[1] += 1
            if i==3:
                count[2] += 1
            if i==4:
                count[3] += 1
            if i==5:
                count[4] += 1

        if (count[0] > thresh) or (count[1] > thresh) or (count[2] > thresh) or (count[3] > thresh) or (count[4] > thresh):
            pc.drop(user, inplace=True)

    #PEOPLE TO KEEP
    keep = list(pc.index)

    pivot = data.pivot_table(index = "UserName", columns = ["Category", "SubCategory", "QuestionId"], values = "Marks")
    pivot.columns.names = (None, None, None)

    #KEEPING APPROVED USERS
    for user, item in pivot.iterrows():
        if user not in keep:
            pivot.drop(user, inplace= True)
            
    pivot.fillna(0, inplace=True)
    return pivot

In [4]:
def normal(db):
    
    print('\n\n\033[1m- NORMS -\033[0m')
    
    dft = pd.DataFrame.copy(db)
    
    print("\n\n\033[1mNumber of Students = \033[0m", len(dft.index))
    
    heads = list(dft.columns.levels[0])
    lvls = list(dft.columns.levels[1])
       
    final_result = pd.DataFrame(index = dft.index, columns = ['remove1', 'remove2'])
    
    final_result.loc['Mean'] = 0
    final_result.loc['Std Dev'] = 0
    
    for j in heads:
        
        for k in lvls:
            
            df = dft[j][k]
            
            
            if not list(df.columns):
                continue
            
            
            original = df.sum(axis=1)
            marks = original.asobject
            marks.sort()
            marks = list(marks)
            x = np.asarray(marks)

            mu = np.mean(marks)
            sigma = np.std(marks)

            graph = plt.hist(marks, 10, facecolor='white', edgecolor='black', linewidth=1.5)[0]
            plt.clf()

            height = max(graph)

            original = original.to_frame()
            original.columns = ['Marks']

            perf = ["" for i in range(len(marks))]

            for i in range(len(perf)):

                val = original.iloc[i]['Marks']

                if val>(mu-sigma) and val<(mu+sigma):
                    perf[i] = 3

                if val>(mu+sigma) and val<(mu+(2*sigma)):
                    perf[i] = 4

                if val<(mu-sigma) and val>(mu-(2*sigma)):
                    perf[i] = 2

                if val>(mu+(2*sigma)):
                    perf[i] = 5

                if val<(mu-(2*sigma)):
                    perf[i] = 1
            
            perf.append(round(mu))
            perf.append(round(sigma))           
            final_result[k] = perf
            
        df = dft[j]
        
        original = df.sum(axis=1)
        marks = original.asobject
        marks.sort()
        marks = list(marks)
        x = np.asarray(marks)

        mu = np.mean(marks)
        sigma = np.std(marks)

        graph = plt.hist(marks, 10, facecolor='white', edgecolor='black', linewidth=1.5)[0]
        plt.clf()

        height = max(graph)

        plt.figure(figsize=(10,7))
        plt.title(j, fontsize=20)
        plt.hist(marks, 10, facecolor='white', edgecolor='black', linewidth=1.5)
        plt.plot(x, mlab.normpdf(x, mu, sigma)*height*height, c='r')
        plt.axvline(mu, linestyle='dashdot', linewidth=2.5)
        plt.axvline(mu-sigma, linestyle='dashed', linewidth=2.5)
        plt.axvline(mu+sigma, linestyle='dashed', linewidth=2.5)
        plt.axvline(mu-2*sigma, linestyle='dotted', linewidth=3)
        plt.axvline(mu+2*sigma, linestyle='dotted', linewidth=3)
        plt.ylabel('No. of Students', fontsize=15)
        plt.xlabel('Marks', fontsize=15)

        plt.text(s=round(mu), x=mu, y=height, fontsize=15, color='m')
        plt.text(s=round(mu-sigma), x=(mu-sigma), y=height, fontsize=15)
        plt.text(s=round(mu+sigma), x=(mu+sigma), y=height, fontsize=15)
        plt.text(s=round(mu-2*sigma), x=(mu-2*sigma), y=height, fontsize=15)
        plt.text(s=round(mu+2*sigma), x=(mu+2*sigma),y=height, fontsize=15)

        plt.show()

        original = original.to_frame()
        original.columns = ['Marks']

        perf = ["" for i in range(len(marks))]

        for i in range(len(perf)):

            val = original.iloc[i]['Marks']

            if val>(mu-sigma) and val<(mu+sigma):
                perf[i] = 3

            if val>(mu+sigma) and val<(mu+(2*sigma)):
                perf[i] = 4

            if val<(mu-sigma) and val>(mu-(2*sigma)):
                perf[i] = 2

            if val>(mu+(2*sigma)):
                perf[i] = 5

            if val<(mu-(2*sigma)):
                perf[i] = 1
                
        perf.append(round(mu))
        perf.append(round(sigma))
                
        final_result[j] = perf
        
    df = dft

    original = df.sum(axis=1)
    marks = original.asobject
    marks.sort()
    marks = list(marks)
    x = np.asarray(marks)

    mu = np.mean(marks)
    sigma = np.std(marks)

    graph = plt.hist(marks, 10, facecolor='white', edgecolor='black', linewidth=1.5)[0]
    plt.clf()

    height = max(graph)

    plt.figure(figsize=(10,7))
    plt.title('Total', fontsize=20)
    plt.hist(marks, 10, facecolor='white', edgecolor='black', linewidth=1.5)
    plt.plot(x, mlab.normpdf(x, mu, sigma)*height*height, c='r')
    plt.axvline(mu, linestyle='dashdot', linewidth=2.5)
    plt.axvline(mu-sigma, linestyle='dashed', linewidth=2.5)
    plt.axvline(mu+sigma, linestyle='dashed', linewidth=2.5)
    plt.axvline(mu-2*sigma, linestyle='dotted', linewidth=3)
    plt.axvline(mu+2*sigma, linestyle='dotted', linewidth=3)
    plt.ylabel('No. of Students', fontsize=15)
    plt.xlabel('Marks', fontsize=15)

    plt.text(s=round(mu), x=mu, y=height, fontsize=15, color='m')
    plt.text(s=round(mu-sigma), x=(mu-sigma), y=height, fontsize=15)
    plt.text(s=round(mu+sigma), x=(mu+sigma), y=height, fontsize=15)
    plt.text(s=round(mu-2*sigma), x=(mu-2*sigma), y=height, fontsize=15)
    plt.text(s=round(mu+2*sigma), x=(mu+2*sigma),y=height, fontsize=15)

    plt.show()

    original = original.to_frame()
    original.columns = ['Marks']

    perf = ["" for i in range(len(marks))]

    for i in range(len(perf)):

        val = original.iloc[i]['Marks']

        if val>(mu-sigma) and val<(mu+sigma):
            perf[i] = 3

        if val>(mu+sigma) and val<(mu+(2*sigma)):
            perf[i] = 4

        if val<(mu-sigma) and val>(mu-(2*sigma)):
            perf[i] = 2

        if val>(mu+(2*sigma)):
            perf[i] = 5

        if val<(mu-(2*sigma)):
            perf[i] = 1
            
    perf.append(round(mu))
    perf.append(round(sigma))
            
    final_result['Total'] = perf
    
    del final_result['remove1']
    del final_result['remove2']
    
    writer = ExcelWriter('Norms.xlsx')
    final_result.to_excel(writer,'Sheet1')
    writer.save()

In [5]:
def cat_cronbach(af):

    df = pd.DataFrame.copy(af)
    nlvls = df.columns.levels[0]

    ndf = pd.DataFrame()

    for i in nlvls:

        if not list(df[i].columns):
            continue

        ndf = pd.concat([ndf, df[i]], axis = 1)

    ndf['Total'] = ndf.sum(axis=1)

    #Items is list of columns
    items = list(ndf.columns)

    #Index values of respective columns
    num = len(items) - 1   

    #Adding row Variance and recalculating variance
    ndf.loc['Variance'] = ndf.var(axis=0)

    results = pd.DataFrame(columns = ["alpha if removed"])

    if(num>=3):

        for i in range(0, num):

            working = pd.DataFrame.copy(ndf)

            working.drop(items[i], axis=1, inplace=True)

            l=len(working.columns) 

            working[items[l]] = 0

            working.drop('Variance', axis=0, inplace=True)

            #print(working)

            working[items[l]] = working.sum(axis=1)  

            working.loc['Variance'] = working.var(axis=0)

            wi = len(working.columns) - 1

            alpha = (wi/(wi-1)) * (1 - sum(working.loc['Variance'][0:wi])/(working.loc['Variance'][wi]))

            #print(working)

            results.loc[items[i]] = alpha

            #print("Cronbach's alpha if \033[1m", items[i], "\033[0m deleted = ", alpha)

        print(results)

        alpha =  (num/(num-1)) * (1 - sum(ndf.loc['Variance'][0:num])/(ndf.loc['Variance'][num]))
        print("\033[1mCategory Cronbach's Alpha = ", alpha, '\033[0m')

    elif (num==2): 
        print("\n!!\033[1mOnly following two items left.\033[0m : ")
        for i in range(num):
            print('-> ',items[i])
        print("Skipping calculation for item if deleted.")
        alpha =  (num/(num-1)) * (1 - sum(ndf.loc['Variance'][0:num])/(ndf.loc['Variance'][num]))
        print("\033[1mCategory Cronbach's Alpha = ", alpha, '\033[0m')

    elif (num<2): print("!!\033[1mLess than two items left.\033[0m Skipping all calculations.")



In [6]:
def cronbach(af):
    
    print('\n\n')
    print("\033[1m-CRONBACH'S ALPHA-\033[0m")
    
    #Create dataframe for function to work with
    
    dft = pd.DataFrame.copy(af)
    
    heads = list(dft.columns.levels[0])
    lvls = list(dft.columns.levels[1])
    
    
    for j in heads:
        
        print('\n')
        print("\033[1m*", j, "*\033[0m")
        
        cat_cronbach(dft[j])
    
        for k in lvls:

            df = dft[j][k]

            if not list(df.columns):
                continue

            print('\n')
            print('\033[1m', k, '\033[0m')

            df['Total'] = df.sum(axis=1)

            #Items is list of columns
            items = list(df.columns)

            #Index values of respective columns
            num = len(items) - 1   

            #Adding row Variance and recalculating variance
            df.loc['Variance'] = df.var(axis=0)

            results = pd.DataFrame(columns = ["alpha if removed"])

            if(num>=3):

                for i in range(0, num):

                    working = pd.DataFrame.copy(df)

                    working.drop(items[i], axis=1, inplace=True)

                    l=len(working.columns) 

                    working[items[l]] = 0

                    working.drop('Variance', axis=0, inplace=True)

                    #print(working)

                    working[items[l]] = working.sum(axis=1)  

                    working.loc['Variance'] = working.var(axis=0)

                    wi = len(working.columns) - 1

                    alpha = (wi/(wi-1)) * (1 - sum(working.loc['Variance'][0:wi])/(working.loc['Variance'][wi]))

                    #print(working)

                    results.loc[items[i]] = alpha

                    #print("Cronbach's alpha if \033[1m", items[i], "\033[0m deleted = ", alpha)

                print(results)

                alpha =  (num/(num-1)) * (1 - sum(df.loc['Variance'][0:num])/(df.loc['Variance'][num]))
                print("\033[1mSubcat Cronbach's Alpha = ", alpha, '\033[0m')

            elif (num==2): 
                print("\n!!\033[1mOnly following two items left.\033[0m : ")
                for i in range(num):
                    print('-> ',items[i])
                print("Skipping calculation for item if deleted.")
                alpha =  (num/(num-1)) * (1 - sum(df.loc['Variance'][0:num])/(df.loc['Variance'][num]))
                print("\033[1mSubcat Cronbach's Alpha = ", alpha, '\033[0m')

            elif (num<2): print("!!\033[1mLess than two items left.\033[0m Skipping all calculations.")

In [7]:
def diff(data):
    
    print('\n\n')
    print('\033[1m-DIFFICULTY-\033[0m')
    
    #Find MAX possible value in marks
    
    heads = list(data.columns.levels[0])
    lvls = list(data.columns.levels[1])
    
    marks = []
    
    for i in heads:
        for j in lvls:
            for k in data[i][j].iterrows():
                if not k[1].empty:
                    marks.append(max(k[1]))
    
    correct = max(set(marks))
    
    no_of_items = 0
    
    for i in heads:
        for j in lvls:
            for k in data[i][j]:
                no_of_items += 1
            
    no_of_users = len(pivot.index)

    result = pd.DataFrame(columns = ['Item', 'Difficulty'], index = range(no_of_items))

    m = -1
    
    for i in heads:
        
        for j in lvls:

            for j in data[i][j].iteritems():
                m += 1
                result['Item'][m] = j[0]
                marks_all = j[1]
                count = 0

                for t in marks_all:
                    #print(t)
                    if t==correct:
                        count += 1
                difficulty = count/no_of_users
                result['Difficulty'].iloc[m] = round(difficulty, 4)

    result.set_index('Item', inplace=True)
    result.index.name = None
    print(result.to_string())


In [8]:
def discr(data):
    
    print('\n\n')
    print('\033[1m-DISCRIMINATION-\033[0m')
    
    heads = list(data.columns.levels[0])
    lvls = list(data.columns.levels[1])
    
    for j in heads:
        
        print('\n')
        print('\033[1m*', j, '*\033[0m')
        
        for k in lvls:
                
                df = data[j][k]
                
                if not list(df.columns):
                    continue

                
                print('\n')
                print('\033[1m', k, '\033[0m')

                #Items is list of columns
                items = list(df.columns)

                results = pd.DataFrame(index = items, columns = ["Discrimination"])

                #Index values of respective columns
                num = len(items) - 1   

                for i in range(0, num+1):

                    working = pd.DataFrame.copy(df)

                    marks = working[items[i]].asobject

                    working.drop(items[i], axis=1, inplace=True)

                    total = working.sum(axis=1).asobject

                    disc = round(pearsonr(marks, total)[0], 5)

                    results.loc[items[i]] = disc

                #Adding row Variance and recalculating variance

                print(results)

In [9]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = list(filedialog.askopenfilename(multiple=False))

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"
        
my_button = SelectFilesButton()
my_button

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [19]:
path = my_button.files
path = ''.join(path)

print("\033[1mSelected file : \033[0m", path.split('/')[-1])

raw = pd.read_excel(path)

pivot = clean(raw)

preference = input("\n\nWhich of the following would you like to include in the report: \n\n1. Normal Curve\n2. Cronbach's Alpha\n3. Difficulty\n4. Discrimination\n\n Enter number against preferences separated by spaces. \n")
preference = preference.split()

if ('1' in preference):
    normal(pivot)

if ('2' in preference):
    cronbach(pivot)

if ('3' in preference):
    diff(pivot)

if ('4' in preference):
    discr(pivot)
    
    


Selected file :  DigitalChain.xlsx


Which of the following would you like to include in the report: 

1. Normal Curve
2. Cronbach's Alpha
3. Difficulty
4. Discrimination

 Enter number against preferences separated by spaces. 
0
